## Data Cleaning for reviews.csv

In [1]:
# Dependencies and packages
%reload_ext lab_black

import os
import pandas as pd
import numpy as np
import math as math

In [3]:
# Read Reviews CSV
csv_path = os.path.join("../data/csv/reviews.csv")
df_reviews = pd.read_csv(csv_path)

In [40]:
df_reviews.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03
1,125646,_dirty_,2017-12-21,,4.50,4.50,4.50,4.50,4.50,4.50
2,125646,CJDUBYA,2017-12-21,,4.75,4.75,4.75,4.75,4.75,4.75
3,125646,GratefulBeerGuy,2017-12-20,0% 16 oz can. Funny story: As I finally wal...,4.75,4.75,4.50,4.50,4.50,4.58
4,125646,LukeGude,2017-12-20,Classic TH NEIPA. Overflowing head and bouq...,4.25,4.50,4.25,4.25,4.25,4.31


In [5]:
# Find the DF size pre-cleaning
print(df_reviews.shape)

(9073128, 10)


In [6]:
# Find the number of unique beers in reviews.csv
df_reviews["beer_id"].value_counts()

11757     17160
2093      15947
7971      14927
1093      14915
29619     14292
          ...  
203777        1
126326        1
109950        1
282050        1
8188          1
Name: beer_id, Length: 309542, dtype: int64

In [ ]:
# Find unique users who reviewed
df_reviews["username"].value_counts()

In [7]:
# Find value counts for every column in reviews.csv
df_reviews.apply(lambda x: x.isnull().value_counts())

,beer_id,username,date,text,look,smell,taste,feel,overall,score
False,9073128.0,9069313,9073128.0,9073128.0,5283110,5283110,5283110,5283110,5283110,9073128.0
True,NaN,3815,NaN,NaN,3790018,3790018,3790018,3790018,3790018,NaN


In [41]:
df_clean = df_reviews.copy()

In [42]:
# Delete all the beers that have less than 100 reviews
df_clean = df_clean[df_clean.groupby("beer_id").beer_id.transform("count") > 100].copy()
df_clean.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score
1,125646,_dirty_,2017-12-21,,4.50,4.50,4.50,4.50,4.50,4.50
2,125646,CJDUBYA,2017-12-21,,4.75,4.75,4.75,4.75,4.75,4.75
3,125646,GratefulBeerGuy,2017-12-20,0% 16 oz can. Funny story: As I finally wal...,4.75,4.75,4.50,4.50,4.50,4.58
4,125646,LukeGude,2017-12-20,Classic TH NEIPA. Overflowing head and bouq...,4.25,4.50,4.25,4.25,4.25,4.31
5,125646,jneff33,2017-12-20,,4.50,4.75,5.00,5.00,5.00,4.91


In [43]:
# The algorithm below reduces the memory size of a dataframe
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage of properties dataframe is :", start_mem_usg, " MB")
    NAlist = []  # Keeps track of columns that have missing values filled in.
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings

            # Print current column type
            print("******************************")
            print("Column: ", col)
            print("dtype before: ", props[col].dtype)

            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()

            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all():
                NAlist.append(col)
                props[col].fillna(mn - 1, inplace=True)

            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = props[col] - asint
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)

            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)

            # Print new column type
            print("dtype after: ", props[col].dtype)
            print("******************************")

    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage is: ", mem_usg, " MB")
    print("This is ", 100 * mem_usg / start_mem_usg, "% of the initial size")
    return props, NAlist

In [44]:
# Check Memory Usage
props = df_clean
props, NAlist = reduce_mem_usage(props)
print("_________________")
print("")
print(
    "Warning: the following columns have missing values filled with 'df['column_name'].min() -1': "
)
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 568.8571548461914  MB
******************************
Column:  beer_id
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  look
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  smell
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  taste
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  feel
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  overall
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  score
dtype before:  float64
dtype after:  float32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  387.85715103149414  MB
This 

In [72]:
df_clean = props.copy()

In [73]:
df_clean.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score
1,125646,_dirty_,2017-12-21,,4.50,4.50,4.50,4.50,4.50,4.50
2,125646,CJDUBYA,2017-12-21,,4.75,4.75,4.75,4.75,4.75,4.75
3,125646,GratefulBeerGuy,2017-12-20,0% 16 oz can. Funny story: As I finally wal...,4.75,4.75,4.50,4.50,4.50,4.58
4,125646,LukeGude,2017-12-20,Classic TH NEIPA. Overflowing head and bouq...,4.25,4.50,4.25,4.25,4.25,4.31
5,125646,jneff33,2017-12-20,,4.50,4.75,5.00,5.00,5.00,4.91


In [74]:
# Drop unwanted columns
df_clean = df_clean.drop(["date", "look", "smell", "taste", "feel", "overall"], axis=1)

In [75]:
df_clean.head()

,beer_id,username,text,score
1,125646,_dirty_,,4.50
2,125646,CJDUBYA,,4.75
3,125646,GratefulBeerGuy,0% 16 oz can. Funny story: As I finally wal...,4.58
4,125646,LukeGude,Classic TH NEIPA. Overflowing head and bouq...,4.31
5,125646,jneff33,,4.91


In [50]:
df_clean.dtypes

beer_id      uint32
username     object
text         object
score       float32
dtype: object

In [76]:
# list comprehension to strip each row
df_clean["text"] = [x.strip(" \xa0") for x in df_clean["text"]]

In [78]:
df_clean = df_clean[df_clean["text"] != ""]

In [79]:
df_clean

,beer_id,username,text,score
3,125646,GratefulBeerGuy,0% 16 oz can. Funny story: As I finally walked...,4.58
4,125646,LukeGude,Classic TH NEIPA. Overflowing head and bouquet...,4.31
7,125646,MFMB,Pours a creamy opaque light straw yellow with ...,4.52
13,125646,jngrizzaffi,Pours a cloudy yellow color with a thin foamy ...,4.53
15,125646,PDOR1960,Another great brew from Treehouse!!,4.50
...,...,...,...,...
9073095,58482,smcolw,Large chalky white head. Leaves lots of spotty...,3.58
9073096,58482,firkinhophead,"pours a hazy, almost cloudy, autumn gold. Appa...",3.92
9073097,58482,CHADMC3,Pours a hazy golden/orange to copper with a bi...,3.80
9073110,104824,zookerman182,Appearance is dark dark brown with a half fing...,4.35


In [80]:
# Read Beers CSV
csv_path2 = os.path.join("../data/csv/beers.csv")
df_beers = pd.read_csv(csv_path2)

In [82]:
df_beers = df_beers.rename(columns={"id": "beer_id"})

In [83]:
# Merge the two dataframe
final_df = pd.merge(df_clean, df_beers[["beer_id", "name", "style"]], on="beer_id")
final_df.head()

,beer_id,username,text,score,name,style
0,125646,GratefulBeerGuy,0% 16 oz can. Funny story: As I finally walked...,4.58,Haze,New England IPA
1,125646,LukeGude,Classic TH NEIPA. Overflowing head and bouquet...,4.31,Haze,New England IPA
2,125646,MFMB,Pours a creamy opaque light straw yellow with ...,4.52,Haze,New England IPA
3,125646,jngrizzaffi,Pours a cloudy yellow color with a thin foamy ...,4.53,Haze,New England IPA
4,125646,PDOR1960,Another great brew from Treehouse!!,4.50,Haze,New England IPA


In [88]:
final_df = final_df.dropna(how="any", axis=0)

In [90]:
final_df.to_csv(os.path.join("../data/csv/reviews_NLTK.csv"))

In [85]:
final_df["beer_id"].value_counts()

645       4364
11757     4300
2093      4252
7971      4155
1093      4054
          ... 
78122        6
124684       5
108184       5
111083       5
138902       4
Name: beer_id, Length: 12533, dtype: int64

In [34]:
# We now have 12533 unique beers instead of 309542 prior cleaning.

In [86]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2199604 entries, 0 to 2199603
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   beer_id   uint32 
 1   username  object 
 2   text      object 
 3   score     float32
 4   name      object 
 5   style     object 
dtypes: float32(1), object(4), uint32(1)
memory usage: 100.7+ MB


In [89]:
final_df.apply(lambda x: x.isnull().value_counts())

,beer_id,username,text,score,name,style
False,2196268,2196268,2196268,2196268,2196268,2196268


## BEGIN HERE For further work using this final csv file. 
This is to save time re-running the whole notebook.

In [32]:
# Reloading
path = os.path.join("../data/csv/reviews_NLTK.csv")
df = pd.read_csv(path)

In [33]:
df.drop("Unnamed: 0", axis=1, inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
df.to_csv("../data/csv/reviews_NLTK.csv", index=False)

In [ ]:
df.shape

In [ ]:
df["beer_id"].value_counts()

In [34]:
df = df[df.groupby("beer_id").beer_id.transform("count") > 300].copy()
df["beer_id"].value_counts()

645       4355
11757     4277
2093      4231
7971      4137
1093      4035
          ... 
149331     301
3347       301
1506       301
1332       301
34129      301
Name: beer_id, Length: 1686, dtype: int64

In [35]:
df.shape

(1255440, 6)

In [36]:
df

,beer_id,username,text,score,name,style
0,125646,GratefulBeerGuy,0% 16 oz can. Funny story: As I finally walked...,4.58,Haze,New England IPA
1,125646,LukeGude,Classic TH NEIPA. Overflowing head and bouquet...,4.31,Haze,New England IPA
2,125646,MFMB,Pours a creamy opaque light straw yellow with ...,4.52,Haze,New England IPA
3,125646,jngrizzaffi,Pours a cloudy yellow color with a thin foamy ...,4.53,Haze,New England IPA
4,125646,PDOR1960,Another great brew from Treehouse!!,4.50,Haze,New England IPA
...,...,...,...,...,...,...
1255435,251571,Zorro,Clear amber colored ale with a good sized whit...,3.52,Tropical Torpedo Tropical IPA,American IPA
1255436,251571,MichaelRabdau,"A perfect companion and complement to Torpedo,...",5.00,Tropical Torpedo Tropical IPA,American IPA
1255437,251571,JohnnyMc,Pours a clear golden color. Aroma is grapefrui...,3.31,Tropical Torpedo Tropical IPA,American IPA
1255438,251571,Belekinbeer,I had this yesterday at SN Mills River. Grante...,4.35,Tropical Torpedo Tropical IPA,American IPA


In [46]:
df = df.groupby("beer_id", as_index=False).apply(
    lambda x: x[x.text.str.len() == x.text.str.len().max()]
)

In [47]:
df.shape

(1686, 6)

In [48]:
df.to_csv(os.path.join("../data/csv/reviews_NLTK1686.csv"), index=False)

In [49]:
df

,,beer_id,username,text,score,name,style
0,49451,5,woodychandler,I feel as though The CANQuest (TM) is once mor...,4.13,Amber,Vienna Lager
1,50061,6,rickyleepotts,"I have had this beer before, and all I can rem...",2.96,Turbodog,English Brown Ale
2,51264,7,Jadjunk,#115. I haven't reviewed much by Abita Brewing...,3.67,Purple Haze,Fruit and Field Beer
3,52504,10,bditty187,"From notes Batch #29 Dark brown in hue, damn ...",3.86,Dubbel Ale,Belgian Dubbel
4,53177,17,mikesgroove,"So, going to Grandmas for dinner tonight, and...",3.78,Widmer Hefeweizen,German Hefeweizen
...,...,...,...,...,...,...,...
1681,1166109,191070,transitionbikes93,I dont know why this beer gets such a harsh ra...,4.88,Guinness Nitro IPA,English India Pale Ale (IPA)
1682,1188196,197183,woodychandler,One of the great joys of The CANQuest (tm) is ...,4.37,Citradelic Tangerine IPA,American IPA
1683,904882,202078,fmccormi,Straight pour from a pry-off 12oz bottle to an...,4.44,Enjoy By Unfiltered IPA,American Imperial IPA
1684,1255228,251571,woodychandler,My unplanned SN horizontal reaches its conclus...,4.14,Tropical Torpedo Tropical IPA,American IPA


#### Cleaning Methodology until this point:
1. Beers with less than 300 reviews
2. Duplicated beers - Essentially keeping 1 review text for each beer. 